In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

In [1]:
from data.traffic_signs import TrafficSigns
from torchvision import transforms

dataset_root = '/home/kurikuri/Projects/Datasets/TrafficSigns/'
transform = (transforms.ToTensor(),
             transforms.Resize((448, 448)))

train_dataset = TrafficSigns(root=dataset_root,
                             mode='train',
                             transform=transform)
val_dataset = TrafficSigns(root=dataset_root,
                           mode='val',
                           transform=transform)
test_dataset = TrafficSigns(root=dataset_root,
                            mode='test',
                            transform=transform)

In [2]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce GTX 1070 Ti


In [4]:
from torch.utils.data import DataLoader

batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [33]:
from model.yolov1 import YoloV1
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
from torch.nn import CrossEntropyLoss
from evaluation.metrics import Metric
from evaluation.eval import ClassificationEvaluator

model = YoloV1('../config/darknet-v1.yaml', 15)
optimizer = Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-9)
scheduler = ExponentialLR(optimizer, gamma=0.9)
loss_fn = model.get_loss()

In [34]:
from trainer.supervised import SimpleTrainer

trainer = SimpleTrainer(model, train_loader, val_loader, loss_fn, optimizer)

In [36]:
epochs = 10

for epoch in range(epochs):
    trainer.train()
    scheduler.step()

RuntimeError: stack expects each tensor to be equal size, but got [1, 5] at entry 0 and [2, 5] at entry 7